In [12]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import feedparser
import requests
import datetime
import lxml
import csv

This is the file that includes the steps for us to scrapy the content from BBC News.

### handle missing data
BBC News has several different formats of the web page, so we will use different ways to access them. Particularly, some News only include the video/image content, without any text content. For this situation, we will use their article descriptions as content.

Define a function to access the content of each article. We will pass the article URL addresses into the function, and it will return each full content of articles. This function is based on Beautiful Soup 4.

In [13]:
def request_content(url):
    news_url = url
    data = requests.get(news_url).text
    soup = BeautifulSoup(data, "lxml")
 
 
    article = ''
 
  
    extract = soup.find_all('div', {"class": "css-uf6wea-RichTextComponentWrapper e1xue1i83"})
   
    if extract != []:
      
        for content in extract:
            article = article + ' ' + content.text
 
        return article
 
    else:
        
        try:
            extract = soup.find('div', {"class": "css-83cqas-RichTextContainer e5tfeyi2"}).extract()
            extracts = extract.find_all_next('p')
 
 
            for content in extracts:
                article = article + ' ' + content.text
 
            return article
 
 
        except:
            article = soup.find('meta', attrs={'name': 'description'}).get('content')
            return article


Define a function to create a csv file with the heading: Title, Content, Tag, Time and URL. 

Then we call this function to create a csv file.

In [14]:
def create_csv():
    rules = ['Title','Content','Tag','Time','URL']
    f = open('./bbcnews.csv','a+',newline='',encoding='utf-8')
    csv_writer = csv.writer(f)
    csv_writer.writerow(rules)
    f.close

In [15]:
create_csv()

Define a function to save the data into csv file.

The data includes article title, article content, article tag, updated time and url address.

In [16]:
def save_to_csv(title,content,tag,time,url):
    f = open('./bbcnews.csv','a+',newline='',encoding='utf-8')
    csv_writer = csv.writer(f)
    csv_writer.writerow([title,content,tag,time,url])
 
    f.close()

In this part, we will crawl data from 9 BBC RSS feeds, including BBC-World, BBC-UK, BBC-politics..etc...

My variables of interest are title and contents of articles which I can use statistical means to analyze the number of their words and analyze what are the most common words of the same type in articles.

We can access URL address, updated time, article title, and tag directly from the feed. For the content of the article, we have to use the function which we defined before (request_content) to access. we will pass the article URL address to this function and it will return the full content of this article.
Then we will use save_to_csv function to save the data into the csv file.

In [17]:
RSS_URLS=[
        'http://feeds.bbci.co.uk/news/world/rss.xml',
        'http://feeds.bbci.co.uk/news/uk/rss.xml',
        'http://feeds.bbci.co.uk/news/business/rss.xml',
        'http://feeds.bbci.co.uk/news/politics/rss.xml',
        'http://feeds.bbci.co.uk/news/health/rss.xml',
        'http://feeds.bbci.co.uk/news/education/rss.xml',
        'http://feeds.bbci.co.uk/news/science_and_environment/rss.xml',
        'http://feeds.bbci.co.uk/news/technology/rss.xml',
        'http://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml'
         ]

feeds=[]
for url in RSS_URLS:
    feeds.append(feedparser.parse(url))

for feed in feeds:
    for article in feed['entries']:
        url=article['link']
        time=article['updated']
        title=article['title']
        tag=feed.feed.title
        content=request_content(article['link'])
        save_to_csv(title,content,tag,time,url)